<a href="https://colab.research.google.com/github/stef4k/train-maintenance-data-mining/blob/main/text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Text classification


In [1]:
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
import numpy as np
from collections import Counter
import ast
import pickle
from joblib import Parallel, delayed
from google.colab import files
from sklearn.model_selection import StratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, confusion_matrix
import numpy as np
import pandas as pd
from sklearn.utils import parallel_backend
import threading
from sklearn.model_selection import StratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from imblearn.over_sampling import SMOTE, BorderlineSMOTE, ADASYN, RandomOverSampler
from imblearn.combine import SMOTEENN, SMOTETomek
from gensim.models import Word2Vec
from scipy.optimize import minimize
from sklearn.base import TransformerMixin
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, classification_report
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from imblearn.over_sampling import SMOTE, BorderlineSMOTE, ADASYN, RandomOverSampler
from imblearn.combine import SMOTEENN, SMOTETomek
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from scipy.sparse import csr_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC
from xgboost import XGBClassifier
from tqdm import tqdm
from scipy.optimize import differential_evolution
import numpy as np
import pandas as pd
from copy import deepcopy
import warnings
warnings.filterwarnings("ignore")


Manually remove the first ';' from the first row in csv file

In [2]:
df = pd.read_csv('sncb_data_challenge.csv', delimiter=';', index_col=0)
df.sample(2)

,incident_id,vehicles_sequence,events_sequence,seconds_to_incident_sequence,approx_lat,approx_lon,train_kph_sequence,dj_ac_state_sequence,dj_dc_state_sequence,incident_type
776,4468833,"[1028, 1028, 1028, 1028, 1028, 1028, 1028, 102...","[4394, 1566, 1570, 2742, 4026, 4168, 4140, 398...","[-9711, -9711, -9711, -2451, -2451, -2445, -24...",50.161041,5.266867,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[False, False, False, False, False, False, Fal...","[True, True, True, True, True, True, False, Fa...",11
741,4467497,"[1050, 1050, 1050, 1050, 1050, 1050, 1050, 105...","[2956, 2956, 2956, 4066, 3636, 3658, 2956, 295...","[-14390, -14377, -14367, -14302, -14280, -1428...",50.507166,5.540767,"[82.4, 77.5, 72.5, 2.4, 0.0, 0.0, 60.5, 56.0, ...","[False, False, False, False, False, False, Fal...","[True, True, True, True, True, True, True, Tru...",13


In [3]:
class Word2VecVectorizer(TransformerMixin):
    def __init__(self, size=100, window=5, min_count=1, workers=4):
        self.size = size
        self.window = window
        self.min_count = min_count
        self.workers = workers
        self.w2v_model = None

    def fit(self, X, y=None):
        sentences = [sentence.split() for sentence in X]
        self.w2v_model = Word2Vec(sentences, vector_size=self.size, window=self.window,
                                  min_count=self.min_count, workers=self.workers)
        return self

    def transform(self, X, y=None):
        transformed_data = np.array([
            np.mean([self.w2v_model.wv[word] for word in sentence.split() if word in self.w2v_model.wv]
                    or [np.zeros(self.
                                 size)], axis=0)
            for sentence in X
        ])
        return csr_matrix(transformed_data)

    def fit_transform(self, X, y=None):
        self.fit(X, y)
        return self.transform(X, y)

In [4]:
class EnsembleModel:
    def __init__(self, trained_models, trained_vectorizers):

        self.trained_models = trained_models
        self.trained_vectorizers = trained_vectorizers
        self.optimized_weights = None

    def _generate_prediction_matrix(self, X):
        predictions = {}
        for (vect_name, samp_name, clf_name), model in self.trained_models.items():
            vectorizer = deepcopy(self.trained_vectorizers[vect_name])
            X_vect = vectorizer.transform(X).toarray()

            predictions[(vect_name, samp_name, clf_name)] = model.predict_proba(X_vect)

        return np.stack(list(predictions.values()), axis=2)

    def fit(self, X_train, y_train):
        pred_matrix = self._generate_prediction_matrix(X_train)

        def fitness(weights):
            weighted_pred = np.tensordot(pred_matrix, weights, axes=([2], [0]))
            final_pred = np.argmax(weighted_pred, axis=1)
            return -f1_score(y_train, final_pred, average='weighted')

        num_models = pred_matrix.shape[2]
        bounds = [(0, 1)] * num_models
        result = differential_evolution(fitness, bounds)

        self.optimized_weights = result.x

    def predict(self, X):
        if self.optimized_weights is None:
            raise ValueError("The ensemble model must be trained using `train_ensemble` before prediction.")

        pred_matrix = self._generate_prediction_matrix(X)
        weighted_pred = np.tensordot(pred_matrix, self.optimized_weights, axes=([2], [0]))
        ensemble_pred = np.argmax(weighted_pred, axis=1)
        return ensemble_pred


In [5]:
class Representations:
  def __init__(self, df):
    self.df = df

  def representation_a(self, df):
            events_types_dict = {}
            for events_sequence in df['events_sequence']:
                row_list = ast.literal_eval(events_sequence)
                unique_events = set(row_list)
                for event in unique_events:
                    if not events_types_dict.get(event):
                        events_types_dict[event] = 0
                    events_types_dict[event] += 1

            sorted_dict = dict(sorted(events_types_dict.items(), key=lambda item: item[1], reverse=True))
            sorted_events_perc_df = pd.DataFrame(list(sorted_dict.items()), columns=['event_type', 'frequency'])
            sorted_events_perc_df['percentage'] = sorted_events_perc_df['frequency'] / df.shape[0] * 100
            sorted_events_perc_df['event_type'] = sorted_events_perc_df['event_type'].astype(str)

            events_low_frequency = list(map(int, list(sorted_events_perc_df[sorted_events_perc_df.percentage <= 85].event_type)))

            df['clean_events_sequence'] = (
                df['events_sequence']
                .apply(ast.literal_eval)
                .apply(lambda x: [i for i in x if i in events_low_frequency])
                .astype(str)
                .replace(r'[\[\],]', '', regex=True)
            )

            self.y = df['incident_type'].copy()
            self.X = df['clean_events_sequence'].copy()
            return self.X, self.y

  def representation_b(self, df):

    before_incident = []
    after_incident = []

    for _, row in df.iterrows():
        events = ast.literal_eval(row['events_sequence'])
        seconds = ast.literal_eval(row['seconds_to_incident_sequence'])
        incident_type = row['incident_type']

        before_events = " ".join([str(event) for event, time in zip(events, seconds) if time <= 0])
        if before_events:
            before_incident.append({
                "events_sequence": before_events,
                "class": incident_type
            })

        after_events = " ".join([str(event) for event, time in zip(events, seconds) if time > 0])
        if after_events:
            after_incident.append({
                "events_sequence": after_events,
                "class": 100
            })

    before_df = pd.DataFrame(before_incident)
    after_df = pd.DataFrame(after_incident)

    return before_df, after_df


  def representation_c(self, df, sequence_length=30):

    overlapping_sequences = []
    step = sequence_length // 2

    for _, row in df.iterrows():
        events = ast.literal_eval(row['events_sequence'])
        seconds = ast.literal_eval(row['seconds_to_incident_sequence'])
        incident_type = row['incident_type']

        for i in range(0, len(events) - sequence_length + 1, step):
            sequence = [str(i) for i in events[i:i + sequence_length]]
            seconds_slice = seconds[i:i + sequence_length]
            sequence_class = incident_type

            overlapping_sequences.append({"sequence": " ".join(sequence), "class": sequence_class})

    sequences_df = pd.DataFrame(overlapping_sequences)
    return sequences_df.sequence, sequences_df['class']


In [6]:

class Experiment:
    def __init__(self, X, y):
        self.X = X
        self.le = LabelEncoder()
        self.y = self.le.fit_transform(y)
        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(X, y, test_size=0.2, random_state=7, stratify=y)
        self.y_train = self.le.transform(self.y_train)
        self.y_test = self.le.transform(self.y_test)
        self.trained_models = {}
        self.trained_vectorizers = {}
        self.trainer_samplers = {}
        self.results = []
        self.sampling_strategies = {
            "SMOTE": SMOTE(sampling_strategy='auto', random_state=1, k_neighbors=3),
            "Borderline-SMOTE": BorderlineSMOTE(sampling_strategy='auto', random_state=1, k_neighbors=3),
            "ADASYN": ADASYN(sampling_strategy='auto', random_state=1, n_neighbors=3),
            "RandomOversampler": RandomOverSampler(sampling_strategy='auto', random_state=1),
            "SMOTE-ENN": SMOTEENN(sampling_strategy='auto', random_state=1),
            "SMOTE-Tomek": SMOTETomek(sampling_strategy='auto', random_state=1)
        }

        self.vectorizers = {
            "TFIDF": TfidfVectorizer(),
            "Count": CountVectorizer(),
            "Word2Vec": Word2VecVectorizer(size=100, window=5, min_count=1)
        }
        self.classifiers = {
            'LogisticRegression': LogisticRegression(),
            'DecisionTree': DecisionTreeClassifier(),
            'RandomForest': RandomForestClassifier(),
            'ExtraTreesClassifier': ExtraTreesClassifier(),
            'GradientBoostingClassifier': GradientBoostingClassifier(),
            'AdaBoostClassifier': AdaBoostClassifier(),
            'GaussianNB': GaussianNB(),
            'KNN': KNeighborsClassifier(),
            'SVM': SVC(probability=True),
            'XGBoost': XGBClassifier(objective="multi:softmax", num_class=len(np.unique(y)), eval_metric="mlogloss", use_label_encoder=False, n_jobs = 2),
        }


    def duplicate_minor_classes(self, X, y, min_instances=5):
        X = X.reset_index(drop=True)
        y  = y.reset_index(drop=True)
        class_counts = y.value_counts()

        minor_classes = class_counts[class_counts < min_instances].index

        minor_class_rows = X.loc[y.isin(minor_classes)]
        minor_class_labels = y.loc[y.isin(minor_classes)]

        duplicated_X = pd.concat([minor_class_rows] * 2, ignore_index=True)
        duplicated_y = pd.concat([minor_class_labels] * 2, ignore_index=True)

        X_balanced = pd.concat([X, duplicated_X], ignore_index=True)
        y_balanced = pd.concat([y, duplicated_y], ignore_index=True)

        X_balanced.reset_index(drop=True, inplace=True)
        y_balanced.reset_index(drop=True, inplace=True)

        return X_balanced, y_balanced


    def test(self, model, model_name, vectorizer, vectorizer_name, sampler, sampler_name):
        skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)
        accuracies, recalls, precisions, f1s = [], [], [], []

        for train_index, test_index in skf.split(self.X, self.y):
            X_train, X_test = self.X.iloc[train_index], self.X.iloc[test_index]
            y_train, y_test = self.y[train_index], self.y[test_index]
            X_train = vectorizer.fit_transform(X_train).toarray()
            X_test = vectorizer.transform(X_test).toarray()
            X_resampled, y_resampled = sampler.fit_resample(X_train, y_train)
            model.fit(X_resampled, y_resampled)
            y_pred = model.predict(X_test)
            accuracies.append(accuracy_score(y_test, y_pred))
            recalls.append(recall_score(y_test, y_pred, average="weighted"))
            precisions.append(precision_score(y_test, y_pred, average="weighted"))
            f1s.append(f1_score(y_test, y_pred, average="weighted"))

        return [
            model_name,
            vectorizer_name,
            sampler_name,
            np.mean(accuracies),
            np.std(accuracies),
            np.mean(recalls),
            np.std(recalls),
            np.mean(precisions),
            np.std(precisions),
            np.mean(f1s),
            np.std(f1s),
        ]

    def training(self):
        results = []
        self.X_train, self.y_train = self.duplicate_minor_classes(
            self.X_train, pd.Series(self.y_train)
        )
        task_list = []

        for vect_name, vectorizer in self.vectorizers.items():
            for samp_name, sampler in self.sampling_strategies.items():
                for clf_name, model in self.classifiers.items():
                    task_list.append(
                        (clf_name, vect_name, samp_name, deepcopy(model), deepcopy(vectorizer), deepcopy(sampler))
                    )

        progress_bar = tqdm(total=len(task_list))
        with parallel_backend("threading"):
            parallel_results = Parallel(n_jobs=-1)(
                delayed(self._process_task)(task, progress_bar) for task in task_list
            )

        results.extend(parallel_results)
        progress_bar.close()

        # Save results and models
        self.results = pd.DataFrame(
            results,
            columns=[
                "Model",
                "Vectorizer",
                "Sampler",
                "Accuracy Mean",
                "Accuracy Std",
                "Recall Mean",
                "Recall Std",
                "Precision Mean",
                "Precision Std",
                "F1 Mean",
                "F1 Std",
            ],
        )
        return self.results

    def _process_task(self, task, progress_bar):
        clf_name, vect_name, samp_name, model, vectorizer, sampler = task
        result = self.test(
            model=model,
            model_name=clf_name,
            vectorizer=vectorizer,
            vectorizer_name=vect_name,
            sampler=sampler,
            sampler_name=samp_name,
        )
        progress_bar.update(1)
        return result

    def test_ensemble(self):
        skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)
        tasks = []

        for train_index, test_index in skf.split(self.X, self.y):
            X_train, X_test = self.X[train_index], self.X[test_index]
            y_train, y_test = self.y[train_index], self.y[test_index]
            tasks.append((X_train, X_test, y_train, y_test))

        total_tasks = len(tasks)
        progress_bar = tqdm(total=total_tasks, desc="Processing folds")
        lock = threading.Lock()  # Lock to manage updates safely across threads

        def process_with_progress(*args):
            result = self._process_fold(*args)
            with lock:
                progress_bar.update(1)
            return result

        with parallel_backend("threading"):
            results = Parallel(n_jobs=-1)(
                delayed(process_with_progress)(X_train, X_test, y_train, y_test)
                for X_train, X_test, y_train, y_test in tasks
            )

        progress_bar.close()

        accuracies, recalls, precisions, f1s = zip(*results)

        return [
            "Ensemble", "Multiple", "Multiple",
            np.mean(accuracies), np.std(accuracies),
            np.mean(recalls), np.std(recalls),
            np.mean(precisions), np.std(precisions),
            np.mean(f1s), np.std(f1s)
        ]

    def _process_fold(self, X_train, X_test, y_train, y_test):
        fold_trained_models = {}
        fold_trained_vectorizers = {}

        for vectorizer_name, vectorizer_i in self.vectorizers.items():
            vectorizer = deepcopy(vectorizer_i)
            X_train_vect = vectorizer.fit_transform(X_train).toarray()
            X_test_vect = vectorizer.transform(X_test).toarray()
            fold_trained_vectorizers[vectorizer_name] = vectorizer

            for sampler_name, sampler_i in self.sampling_strategies.items():
                sampler = deepcopy(sampler_i)
                X_resampled, y_resampled = sampler.fit_resample(X_train_vect, y_train)

                for model_name, model in self.classifiers.items():
                    trained_model = deepcopy(model)
                    trained_model.fit(X_resampled, y_resampled)
                    fold_trained_models[(vectorizer_name, sampler_name, model_name)] = trained_model

        ensemble = EnsembleModel(fold_trained_models, fold_trained_vectorizers)
        ensemble.fit(X_train, y_train)

        y_pred = ensemble.predict(X_test)

        accuracy = accuracy_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred, average="weighted")
        precision = precision_score(y_test, y_pred, average="weighted")
        f1 = f1_score(y_test, y_pred, average="weighted")

        return accuracy, recall, precision, f1



In [7]:
def duplicate_minor_classes(X, y, min_instances=10):
    X = X.reset_index(drop=True)
    y  = y.reset_index(drop=True)
    class_counts = y.value_counts()

    minor_classes = class_counts[class_counts <= min_instances].index

    minor_class_rows = X.loc[y.isin(minor_classes)]
    minor_class_labels = y.loc[y.isin(minor_classes)]

    duplicated_X = pd.concat([minor_class_rows] * 2, ignore_index=True)
    duplicated_y = pd.concat([minor_class_labels] * 2, ignore_index=True)

    X_balanced = pd.concat([X, duplicated_X], ignore_index=True)
    y_balanced = pd.concat([y, duplicated_y], ignore_index=True)

    X_balanced.reset_index(drop=True, inplace=True)
    y_balanced.reset_index(drop=True, inplace=True)

    return X_balanced, y_balanced

Testing the pipeline with no sequence modification:

In [ ]:
X, y = duplicate_minor_classes(df, df.incident_type)
exp = Experiment(X, y)
results = exp.training()
ensemble = EnsembleModel(exp.trained_models, exp.trained_vectorizers)
ensemble.fit(exp.X_train, exp.y_train)
ensemble_predictions = ensemble.predict(exp.X_test)
print(classification_report(exp.y_test, ensemble_predictions, zero_division=1))

  0%|          | 0/180 [00:00<?, ?it/s]

KeyError: "None of [Index([  0,   1,   2,   3,   4,   6,   7,   8,   9,  10,\n       ...\n       844, 845, 846, 847, 848, 849, 850, 851, 853, 854],\n      dtype='int64', length=684)] are in the [columns]"

In [ ]:
results.sort_values(by=['F1 Mean', "F1 Std"], ascending=False).head(10)

,Model,Vectorizer,Sampler,Accuracy Mean,Accuracy Std,Recall Mean,Recall Std,Precision Mean,Precision Std,F1 Mean,F1 Std
64,GradientBoostingClassifier,Count,SMOTE,0.718129,0.050224,0.718129,0.050224,0.722637,0.054595,0.712874,0.053408
74,GradientBoostingClassifier,Count,Borderline-SMOTE,0.711111,0.039247,0.711111,0.039247,0.720117,0.038706,0.707874,0.040774
84,GradientBoostingClassifier,Count,ADASYN,0.712281,0.050360,0.712281,0.050360,0.717498,0.053334,0.707082,0.052112
114,GradientBoostingClassifier,Count,SMOTE-Tomek,0.702924,0.046549,0.702924,0.046549,0.707647,0.050819,0.697597,0.048697
94,GradientBoostingClassifier,Count,RandomOversampler,0.695906,0.034696,0.695906,0.034696,0.716885,0.038906,0.695975,0.038387
54,GradientBoostingClassifier,TFIDF,SMOTE-Tomek,0.683041,0.044598,0.683041,0.044598,0.727463,0.046621,0.687537,0.046580
34,GradientBoostingClassifier,TFIDF,RandomOversampler,0.684211,0.024253,0.684211,0.024253,0.702683,0.025199,0.684024,0.024986
4,GradientBoostingClassifier,TFIDF,SMOTE,0.676023,0.026824,0.676023,0.026824,0.711289,0.030520,0.678414,0.028567
24,GradientBoostingClassifier,TFIDF,ADASYN,0.677193,0.027529,0.677193,0.027529,0.707741,0.027595,0.678114,0.029618
14,GradientBoostingClassifier,TFIDF,Borderline-SMOTE,0.678363,0.035859,0.678363,0.035859,0.708378,0.030483,0.677870,0.038310


In [8]:
df['events_sequence'] = df['events_sequence'].apply(lambda x: ' '.join([str(i) for i in ast.literal_eval(x)]))

In [9]:
X, y = duplicate_minor_classes(df['events_sequence'], df.incident_type)
exp = Experiment(X, y)
results = exp.training()

100%|██████████| 180/180 [3:54:55<00:00, 78.31s/it] 


In [10]:
results.sort_values(by=['F1 Mean', "F1 Std"], ascending=False).head(10)

,Model,Vectorizer,Sampler,Accuracy Mean,Accuracy Std,Recall Mean,Recall Std,Precision Mean,Precision Std,F1 Mean,F1 Std
74,GradientBoostingClassifier,Count,Borderline-SMOTE,0.737124,0.015696,0.737124,0.015696,0.745760,0.015078,0.734606,0.017097
84,GradientBoostingClassifier,Count,ADASYN,0.736198,0.015580,0.736198,0.015580,0.743790,0.020372,0.734363,0.017607
94,GradientBoostingClassifier,Count,RandomOversampler,0.730578,0.014868,0.730578,0.014868,0.747698,0.016284,0.728193,0.016961
114,GradientBoostingClassifier,Count,SMOTE-Tomek,0.725896,0.016086,0.725896,0.016086,0.734395,0.020848,0.722390,0.017341
64,GradientBoostingClassifier,Count,SMOTE,0.724962,0.017325,0.724962,0.017325,0.732731,0.019960,0.720797,0.020280
24,GradientBoostingClassifier,TFIDF,ADASYN,0.723106,0.023153,0.723106,0.023153,0.748700,0.033380,0.720530,0.021870
54,GradientBoostingClassifier,TFIDF,SMOTE-Tomek,0.713729,0.018439,0.713729,0.018439,0.744645,0.016469,0.712426,0.018053
4,GradientBoostingClassifier,TFIDF,SMOTE,0.712790,0.020132,0.712790,0.020132,0.746933,0.026266,0.710731,0.020392
14,GradientBoostingClassifier,TFIDF,Borderline-SMOTE,0.708135,0.023236,0.708135,0.023236,0.744306,0.023617,0.707300,0.020308
34,GradientBoostingClassifier,TFIDF,RandomOversampler,0.709056,0.022627,0.709056,0.022627,0.727769,0.019977,0.706443,0.019665


In [14]:
ensemble_results = pd.DataFrame(exp.test_ensemble()).T
ensemble_results.columns = ["Model","Vectorizer","Sampler","Accuracy Mean","Accuracy Std","Recall Mean","Recall Std","Precision Mean","Precision Std","F1 Mean","F1 Std"]



Processing folds:  40%|████      | 2/5 [3:06:28<4:39:43, 5594.35s/it]

Processing folds:  20%|██        | 1/5 [3:47:59<15:11:59, 13679.78s/it]

Processing folds:  40%|████      | 2/5 [3:49:43<4:44:41, 5693.94s/it]  

Processing folds:  40%|████      | 2/5 [3:49:36<4:43:54, 5678.14s/it]  

Processing folds:  60%|██████    | 3/5 [5:27:44<3:12:28, 5774.16s/it]

Processing folds:  80%|████████  | 4/5 [5:27:56<58:19, 3499.46s/it]  

Processing folds: 100%|██████████| 5/5 [6:29:43<00:00, 4676.78s/it]


In [15]:
ensemble_results

,Model,Vectorizer,Sampler,Accuracy Mean,Accuracy Std,Recall Mean,Recall Std,Precision Mean,Precision Std,F1 Mean,F1 Std
0,Ensemble,Multiple,Multiple,0.69875,0.022933,0.69875,0.022933,0.712467,0.020184,0.695395,0.021116


In [ ]:
X, y = Representations(df).representation_a(df)
temp = pd.concat([X, y], axis=1)
X, y = duplicate_minor_classes(temp.clean_events_sequence, temp.incident_type)
exp = Experiment(X, y)
results = exp.training()

100%|██████████| 180/180 [2:53:51<00:00, 57.95s/it] 


In [ ]:
results.sort_values(by=['F1 Mean', "F1 Std"], ascending=False).head(10)

,Model,Vectorizer,Sampler,Accuracy Mean,Accuracy Std,Recall Mean,Recall Std,Precision Mean,Precision Std,F1 Mean,F1 Std
94,GradientBoostingClassifier,Count,RandomOversampler,0.705263,0.045987,0.705263,0.045987,0.737284,0.044418,0.706588,0.046902
9,XGBoost,TFIDF,SMOTE,0.707602,0.027179,0.707602,0.027179,0.712896,0.030606,0.701747,0.030119
74,GradientBoostingClassifier,Count,Borderline-SMOTE,0.700585,0.025194,0.700585,0.025194,0.705640,0.031134,0.696156,0.027643
64,GradientBoostingClassifier,Count,SMOTE,0.694737,0.037609,0.694737,0.037609,0.699809,0.036222,0.690855,0.037930
19,XGBoost,TFIDF,Borderline-SMOTE,0.697076,0.028264,0.697076,0.028264,0.704267,0.036849,0.690511,0.029788
39,XGBoost,TFIDF,RandomOversampler,0.694737,0.033777,0.694737,0.033777,0.701423,0.038474,0.687077,0.036004
59,XGBoost,TFIDF,SMOTE-Tomek,0.688889,0.040583,0.688889,0.040583,0.690905,0.043359,0.681567,0.042450
4,GradientBoostingClassifier,TFIDF,SMOTE,0.683041,0.024921,0.683041,0.024921,0.707413,0.024399,0.680730,0.029407
54,GradientBoostingClassifier,TFIDF,SMOTE-Tomek,0.681871,0.039766,0.681871,0.039766,0.711440,0.044594,0.680490,0.043777
99,XGBoost,Count,RandomOversampler,0.686550,0.026309,0.686550,0.026309,0.685470,0.026368,0.678221,0.026599


In [ ]:
ensemble_results = pd.DataFrame(exp.test_ensemble()).T
ensemble_results.columns = ["Model","Vectorizer","Sampler","Accuracy Mean","Accuracy Std","Recall Mean","Recall Std","Precision Mean","Precision Std","F1 Mean","F1 Std"]


Processing folds:   0%|          | 0/5 [1:27:10<?, ?it/s]

Processing folds: 100%|██████████| 5/5 [4:14:51<00:00, 3058.21s/it]


In [ ]:
ensemble_results

,Model,Vectorizer,Sampler,Accuracy Mean,Accuracy Std,Recall Mean,Recall Std,Precision Mean,Precision Std,F1 Mean,F1 Std
0,Ensemble,Multiple,Multiple,0.715642,0.018258,0.715642,0.018258,0.721418,0.017787,0.710738,0.020569


In [ ]:
df_before, df_after = Representations(df).representation_b(df)
X, y = pd.concat([df_before.events_sequence, df_after.events_sequence]), pd.concat([df_before['class'], df_after['class']])
exp = Experiment(X, y)
results = exp.training()
ensemble = EnsembleModel(exp.trained_models, exp.trained_vectorizers)
ensemble.fit(exp.X_train, exp.y_train)
ensemble_predictions = ensemble.predict(exp.X_test)
print(classification_report(exp.y_test, ensemble_predictions, zero_division=1))

In [ ]:
results

In [ ]:
X, y = Representations(df).representation_c(df, sequence_length=100)
exp = Experiment(X, y)
results = exp.training()
ensemble = EnsembleModel(exp.trained_models, exp.trained_vectorizers)
ensemble.fit(exp.X_train, exp.y_train)
ensemble_predictions = ensemble.predict(exp.X_test)
print(classification_report(exp.y_test, ensemble_predictions, zero_division=1))

In [ ]:
results